# Photoreceptor segmentation using [napari](https://napari.org/) and [cellpose](https://cellpose-napari.readthedocs.io/en/latest/index.html)

### Basic usage:
```python
viewer = napari.Viewer()
nbscreenshot(viewer)
viewer.close()
```

example notebooks [here](https://github.com/sofroniewn/napari-training-course/blob/master/lessons/)

In [2]:
# !mamba install -c conda-forge devbio-napari --yes
# !jupyter labextension list
# !mamba install -c conda-forge cellpose-napari --yes
# from cellpose import models

ModuleNotFoundError: No module named 'cellpose'

In [ ]:
# image analysis
import napari
from magicgui import magicgui
from enum import Enum
import cellpose_napari
import cellpose
from cellpose import models
from tifffile import imread
import napari_nikon_nd2
from skimage.io import imread
from magicgui import magicgui
# python basics and plotting
import numpy as np
import pandas as pd
import math
from scipy import ndimage
from scipy.stats import kruskal
from scipy.stats import zscore
from scipy.stats import sem
import scikit_posthocs
import scipy.spatial
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import svgutils
from svgutils.compose import *
# system interaction
import os.path
import time, sys, warnings, re
from tqdm.notebook import tqdm, trange
import importlib
import ipywidgets
from IPython.display import display
# lab brews
from plotParams import *
from lloyd import *


def getFileList(dirData,gene,fileNameMatch,addDetails=''):
   fileList = sorted([f for f in os.listdir(dirData) if not f.startswith('.')]) # get all the files in the data directory (excluding system files) and sort them alphabatically
   fileList = ([f for f in fileList if gene in f]) # select files that match the gene of interest
   fileList = ([f for f in fileList if any([s in f for s in fileNameMatch])]) # only keep relevant files
   if (addDetails=='path'):
      fileList = (['# filePath = \'' + f for f in fileList]) # add text before each file
      fileList = list(map(lambda st: str.replace(st, '.nd2', '\'; gene = \'' + gene + '\'; '), fileList)) #remove file extension (.nd2) and add text after each file
   elif (addDetails=='list'):
      fileList = (['\'' + f for f in fileList]) # add text before each file
      fileList = list(map(lambda st: str.replace(st, '.nd2', '\','), fileList)) #remove file extension (.nd2) and add text after each file
   else:
      fileList = list(map(lambda st: str.replace(st, '.nd2', ''), fileList)) #remove file extension (.nd2) and add text after each file
   return fileList


pBGMenu = ipywidgets.Dropdown(options=['Light', 'Dark'],value='Dark',disabled=False, layout=ipywidgets.Layout(width='20%', height='40px'))
pBGLabel = ipywidgets.widgets.Label('Select plotting style', layout=ipywidgets.Layout(width='20%', height='30px'))
pBG = ipywidgets.widgets.VBox([pBGLabel,pBGMenu])
pBG

In [37]:
applyPlotStyle(pBGMenu.value)
viewer = napari.Viewer()

Plotting style is Dark


In [8]:
viewer = napari.Viewer()

In [35]:
        viewer.close()

***
# Index <a id='Index'></a>
***
- [define directories](#dirDef)
- [Extraction of z planes for analysis](#zExtract)
- [Segmentation with cellpose](#cellSeg)
- [Manual correction of segmentation](#manualCuration)
- [Create thumbnails _WIP_](#thumbnails)
- [Quantification](#quantification)

***
## Define directories<a id='dirDefdirDef'>∮</a>
***
[Back to Index](#Index)

In [3]:
# define where all the imaging data is stored
dirData = 'E:/Imaging/AXR/20240726_F0CRX2b_s1c_xOG/'
# define where all the analysis files should be stored
# subdirectories for each gene and transgenic lines need to be manually made first
dirAnalysis = "E:/Analysis/F0CRXTbx2b/xOGs1C/"

In [4]:
gene = 'inj' # usually 'wt' or gene of interest (e.g. 'tbx2a', 'lhx1a')
# gene = 'lhx1a' # usually 'wt' or gene of interest (e.g. 'tbx2a', 'lhx1a')
fileNameMatch = ['002','004'] # characters in filenames that define zoom-in files

# get a list with all the relevant files
print(*getFileList(dirData,gene,fileNameMatch,addDetails='path'), sep = "\n") #display the file list for copy and paste below

# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_002'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_004'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_004_mip'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L02_002'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L03_002'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L01_002'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_002'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_004'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L03_002'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L01_002'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_002'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_004'; gene = 'inj'; 
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L03_002'; gene = 'inj'; 
# filePath = '20240726_F0C

***
## Extract layers from z-stacks<a id='zExtract'>∮</a>
> Open z-stack &rarr; Pick limits in z for Maximum-Intensity Projection (_MIP_) &rarr; save MIPs &rarr; Ensure image was located in central retina
***
[Back to Index](#Index)

In [27]:
# clear viewer if needed
viewer.layers.select_all()
viewer.layers.remove_selected()

In [5]:
# clear viewer before any loading
viewer.layers.select_all()
viewer.layers.remove_selected()

# open files one by one (by commenting and uncommenting each one)

filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L01_002'; gene = 'uninj'; zlims = [11,21];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_002'; gene = 'uninj'; zlims = [10,22];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_004'; gene = 'uninj'; zlims = [11,20];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L03_002'; gene = 'uninj'; zlims = [9,24];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_002'; gene = 'uninj'; zlims = [16,21];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_004'; gene = 'uninj'; zlims = [16,19];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_002'; gene = 'uninj'; zlims = [2,28];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_004'; gene = 'uninj'; zlims = [9,19];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_002'; gene = 'uninj'; zlims = [16,25];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_004'; gene = 'uninj'; zlims = [11,23]; 

# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_002'; gene = 'inj'; zlims = [14,19];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_004'; gene = 'inj'; #EXCLUDE 
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_004_mip'; gene = 'inj'; #EXCLUDE
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L02_002'; gene = 'inj'; zlims = [15,19];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L03_002'; gene = 'inj'; zlims = [19,40];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L01_002'; gene = 'inj'; zlims = [24,31];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_002'; gene = 'inj'; zlims = [15,22];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_004'; gene = 'inj'; zlims = [11,19];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L03_002'; gene = 'inj'; zlims = [12,20];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L01_002'; gene = 'inj'; zlims = [12,21];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_002'; gene = 'inj'; zlims = [19,23];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_004'; gene = 'inj'; zlims = [7,16];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L03_002'; gene = 'inj'; zlims = [7,20];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L04_002'; gene = 'inj'; zlims = [14,20];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L05_002'; gene = 'inj'; zlims = [16,18];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L06_002'; gene = 'inj'; zlims = [14,17];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_002'; gene = 'inj'; zlims = [13,17];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_004'; gene = 'inj'; zlims = [13,25];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_002'; gene = 'inj'; zlims = [3,27];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_004'; gene = 'inj'; zlims = [13,18];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_002'; gene = 'inj'; zlims = [19,22];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_004'; gene = 'inj'; zlims = [15,24];


layerNames = ['R', 'U', 'N']
# layerNames = ['N', 'U']

# create subdirectory for analysis output
dirOut = dirAnalysis + gene + '/' + filePath + '/'
if (os.path.isdir(dirOut)==False):
   os.mkdir(dirOut)
   print('Created output directory')

print('Canvas cleared. Loading data...\n\n')

# load whole stack to figure out best z-planes for cellpose
viewer.open((dirData+filePath + '.nd2'), plugin='napari-nikon-nd2')
viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();
if len(viewer.layers)==2: #no DAPI
   nChannels = 2
   viewer.layers[0].colormap = 'cyan'
   viewer.layers[0].name = layerNames[0]
   viewer.layers[1].colormap = 'magenta'
   viewer.layers[1].name = layerNames[1]
elif len(viewer.layers)==3: #DAPI
   nChannels = 3
   viewer.layers[0].colormap = 'cyan'
   viewer.layers[0].name = 'N'
   viewer.layers[0].opacity = 0.99
   viewer.layers[1].colormap = 'green'
   viewer.layers[1].name = layerNames[0]
   viewer.layers[2].colormap = 'magenta'
   viewer.layers[2].name = layerNames[1]
   layerNames.append('N')
elif len(viewer.layers)==4: #DAPI
   nChannels = 4
   viewer.layers[0].colormap = 'gray'
   viewer.layers[0].name = 'N'
   viewer.layers[0].opacity = 0.99
   viewer.layers[1].colormap = 'green'
   viewer.layers[1].name = layerNames[0]
   viewer.layers[2].colormap = 'magenta'
   viewer.layers[2].name = layerNames[1]
   viewer.layers[3].colormap = 'bop orange'
   viewer.layers[3].name = layerNames[2]
   layerNames.append('N')

print('Images loaded!')

Canvas cleared. Loading data...


Images loaded!


In [18]:
# make mips (and remove any previous ones) by defining the z-limits to use
zlims = [15,24];
if len(viewer.layers)>nChannels:
    for l in viewer.layers[nChannels:]:
      viewer.layers.remove(l)

for l in viewer.layers[0:nChannels]:
    l.visible = False
#     viewer.layers.remove(l.name + '_mip')
    viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

In [19]:
# SAVE in folder for batch cellpose analysis (when mips look good)
for name in layerNames:
   l = viewer.layers[name + '_mip']; l.save(dirOut + l.name + '.tiff')
print('MIP layers saved for ' + filePath)

MIP layers saved for 20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_004


### Establish landmarks in whole-eye MIP
- Use eyeMarks layer to mark:
    - [ ] Landmark #1: Center of FOV (where zoom in is centered)
    - [ ] Landmark #2: Center of Optic Nerve Head 
    - [ ] Landmark #3: Dorsal edge of eye
    - [ ] Landmark #4: Ventral edge of eye
    - [ ] Landmark #5: Temporal edge of eye (strike zone side)
    - [ ] Landmark #6: Nasal edge of eye
- Decide if stack should be excluded (and stop analysis here and make note in csv summary file)

In [20]:
nMIP = len(layerNames)
for name in layerNames:
   lName = name + "_mip"
   viewer.layers[lName].scale = [1.5/8,1.5/8];
   viewer.layers[lName].translate = [0,+(512-1024*1.5/(8*2))]

# Load whole-eye MIP to establish landmarks
print('Loading eye mip...')
if str.endswith(filePath,'002'):
    mipPath = filePath[:-1] + '1_mip';
elif str.endswith(filePath,'004'):
    mipPath = filePath[:-1] + '3_mip';
elif str.endswith(filePath,'b'):
    mipPath = filePath[:-1] + 'a_mip';
elif str.endswith(filePath,'d'):
    mipPath = filePath[:-1] + 'c_mip';
viewer.open((dirData+mipPath + '.nd2'), plugin='napari-nikon-nd2', blending='additive')

viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();
eyeChannels = 0
if len(viewer.layers)==2+nMIP+nChannels: # no DAPI, GFP and RFP
   eyeChannels = 2
   viewer.layers[nMIP+nChannels].colormap = 'green'
   viewer.layers[nMIP+nChannels].name = 'eyeGreen'
   viewer.layers[nMIP+nChannels+1].colormap = 'magenta'
   viewer.layers[nMIP+nChannels+1].name = 'eyeRed'
elif len(viewer.layers)==3+nMIP+nChannels: # leave DIC
   eyeChannels = 3
   viewer.layers[nMIP+nChannels+0].colormap = 'green'
   viewer.layers[nMIP+nChannels+0].name = 'eyeGreen'
   viewer.layers[nMIP+nChannels+1].colormap = 'magenta'
   viewer.layers[nMIP+nChannels+1].name = 'eyeRed'
   viewer.layers[nMIP+nChannels+2].colormap = 'gray'
   viewer.layers[nMIP+nChannels+2].name = 'eyeBlue'
   viewer.layers[nMIP+nChannels+2].opacity = 0.33
elif len(viewer.layers)==4+nMIP+nChannels: #DIC + DAPI
   eyeChannels = 4
   viewer.layers[nMIP+nChannels+0].colormap = 'gray'
   viewer.layers[nMIP+nChannels+0].name = 'eyeBlue'
   viewer.layers[nMIP+nChannels+1].colormap = 'green'
   viewer.layers[nMIP+nChannels+1].name = 'eyeGreen'
   viewer.layers[nMIP+nChannels+2].colormap = 'magenta'
   viewer.layers[nMIP+nChannels+2].name = 'eyeRed'
   viewer.layers[nMIP+nChannels+3].colormap = 'bop orange'
   viewer.layers[nMIP+nChannels+3].name = 'eyePNA'
   viewer.layers[nMIP+nChannels+3].opacity = 0.33
print(eyeChannels)
print('Loaded: ' + filePath)
eyeMarks = viewer.add_points(size=20, name = 'eyeMarks', symbol='cross', face_color='#ffffff80')
viewer.layers.select_next();

# Save eye landmarks
eyeMarksButton = ipywidgets.Button(description='Save eyeMarks', layout=ipywidgets.Layout(width='200px', height='100px'))
out = ipywidgets.Output()
def eyeMClick(_):
   with out:
      l = viewer.layers['eyeMarks']; l.save(dirOut + l.name + '.csv');
      print('eyeMarks layers saved for ' + filePath)
   # clear viewer
   viewer.layers.select_all()
   viewer.layers.remove_selected()
   eyeMarksButton.disabled = True # to prevent double clicking warnings. All sales are final!

eyeMarksButton.on_click(eyeMClick)

print('Make 6 landmark points then click button to save')

ipywidgets.VBox([eyeMarksButton,out])

Loading eye mip...


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Imaging\\AXR\\20240726_F0CRX2b_s1c_xOG\\20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_003_mip.nd2'

### [Go back and run next one &uarr;](#zExtract)

***
## Segmentation using cellpose<a id='cellSeg'>∮</a>
> Define list of files to segment &rarr; Define cellpose parameters &rarr; Run cellpose in each image
***
[Back to Index](#Index)

### Running cellpose as batch analysis
Usually for:
- Rods: diameter = 20; flow_threshold = 0.5 and mask_threshold = 0.0
- Cones: diameter = 40; flow_threshold = 0.5 and mask_threshold = 0.0
- Horizontal cells: diameter = 100; flow_threshold = 0.6 and mask_threshold = 0.0

> Using a higher _flow_threshold_ (i.e. 0.6 instead of 0.4) will provide more ROIs

In [21]:
# get file list again
gene = 'inj'
print(*getFileList(dirData,gene,fileNameMatch,addDetails='list'), sep = "\n") #display the file list for copy and paste below

'20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_002',
'20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_004',
'20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_004_mip',
'20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L02_002',
'20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L03_002',
'20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L01_002',
'20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_002',
'20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_004',
'20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L03_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L01_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_004',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L03_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L04_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L05_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L06_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_004',
'20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_004',
'20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_002',
'20240726_F0

In [14]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

# collect file paths for injected
# gene_wt = 'uninj'
# filePaths_wt = [
# '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L01_002',
# '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_002',
# '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_004',
# '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L03_002',
# '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_002',
# '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_004',
# '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_002',
# '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_004',
# '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_002',
# '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_004',
#          ]

# collect file paths for uninjected
gene_F0 = 'inj'
filePaths_F0 = [
#'20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_002', #completed
#'20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_004',
#'20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_004_mip',
#'20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L02_002', #no r mip, ignore or go back and get UV
'20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L03_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L01_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_004',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L03_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L04_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L05_002',
'20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L06_002',
         ]


# define cellpose parameters
cpParams = {
    'model' : 'cyto2', # default is 'cyto' or 'cyto2'
    'net_avg' : True,
    'channels' : [0,0], #single channel without nucleus info
    'diameterR' : 25,
    'diameterC' : 40,
    'diameterH' : 100,
    'flow_threshold' : 0.5,
    'cellprob_threshold' : 0.0
}


# define model to use (e.g. 'cyto2')
model = models.Cellpose(gpu=False, model_type=cpParams['model'])

print('I have now defined the new files to run')

I have now defined the new files to run


#### Run in loop to track progress and automatically save results
> In Jan 2022: ~4 minutes per 1024 x 1024 image (4 nets). 

In [27]:
layerNames = ['R','U']

In [15]:
# print('Starting analysis for '+ gene_wt + ':')
# with tqdm(total=len(filePaths_wt)*len(layerNames), file=sys.stdout) as progBar:
#    for fp in filePaths_wt:
#       print('\t' + fp)
#       for l in layerNames:
#          print('\t\t ' + l)
#          startTime = time.time()
#          imgPath = dirAnalysis + gene_wt + '/' + fp + '/' + l + '_mip.tiff'
#          img = cellpose.io.imread(imgPath)
#          if l=='R':
#             diameterCell = cpParams['diameterR']
#          else:
#             diameterCell = cpParams['diameterC']
#          masks, flows, styles, diams = model.eval(img, 
#                                                 diameter=diameterCell, channels=cpParams['channels'],
#                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
#                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
#          cellpose.io.masks_flows_to_seg(img, masks, flows, diams, imgPath, cpParams['channels']) # save results
#          endTime = time.time()
#          print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
#          progBar.update(1)

# print('Finished cellpose batch analysis for '+ gene_wt)

print('Starting analysis for '+ gene_F0 + ':')
with tqdm(total=len(filePaths_F0)*len(layerNames), file=sys.stdout) as progBar:
   for fp in filePaths_F0:
      print('\t' + fp)
      for l in layerNames:
         print('\t\t ' + l)
         startTime = time.time()
         imgPath = dirAnalysis + gene_F0 + '/' + fp + '/' + l + '_mip.tiff'
         img = cellpose.io.imread(imgPath)
         if l=='R':
            diameterCell = cpParams['diameterR']
         else:
            diameterCell = cpParams['diameterC']
         masks, flows, styles, diams = model.eval(img, 
                                                diameter=diameterCell, channels=cpParams['channels'],
                                                do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
         cellpose.io.masks_flows_to_seg(img, masks, flows, diams, imgPath, cpParams['channels']) # save results
         endTime = time.time()
         print ('\t\t\t Time elapsed: {elapsedTime} s'.format(elapsedTime = int(endTime - startTime)))
         progBar.update(1)

print('Finished cellpose batch analysis for '+ gene_F0)

Starting analysis for inj:


  0%|          | 0/32 [00:00<?, ?it/s]

	20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L03_002
		 R
			 Time elapsed: 82 s
		 U
			 Time elapsed: 37 s
		 N
			 Time elapsed: 35 s
		 N
			 Time elapsed: 37 s
	20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L01_002
		 R
			 Time elapsed: 83 s
		 U
			 Time elapsed: 41 s
		 N
			 Time elapsed: 45 s
		 N
			 Time elapsed: 49 s
	20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_002
		 R
			 Time elapsed: 86 s
		 U
			 Time elapsed: 49 s
		 N
			 Time elapsed: 56 s
		 N
			 Time elapsed: 60 s
	20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_004
		 R
			 Time elapsed: 80 s
		 U
			 Time elapsed: 39 s
		 N
			 Time elapsed: 37 s
		 N
			 Time elapsed: 38 s
	20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L03_002
		 R
			 Time elapsed: 85 s
		 U
			 Time elapsed: 48 s
		 N
			 Time elapsed: 41 s
		 N
			 Time elapsed: 41 s
	20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L04_002
		 R
			 Time elapsed: 86 s
		 U
			 Time elapsed: 37 s
		 N
			 Time elapsed: 39 s
		 N
			 Time elapsed: 37 s
	20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L05_002
		 R
			 Time elapse

***
## Manual correction of cellpose segmentation<a id='manualCuration'>∮</a>
> Copy file list from "Extract Layers" &rarr; Open a single file and manually fix ("_curate_") cellpose segmentation

WARNINGS:
- In tg[_mws2_:GFP], a subset of S cones are GFP+. If available, use tg[_sws2_:mCherry] to delete those labels from the M-cone segmentation
***
[Back to Index](#Index)

In [47]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

# uninj files
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L01_002'; gene = 'uninj'; zlims = [11,21];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_002'; gene = 'uninj'; zlims = [10,22];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_004'; gene = 'uninj'; zlims = [11,20];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L03_002'; gene = 'uninj'; zlims = [9,24];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_002'; gene = 'uninj'; zlims = [16,21];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_004'; gene = 'uninj'; zlims = [16,19];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_002'; gene = 'uninj'; zlims = [2,28]; #too curved to analyze
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_004'; gene = 'uninj'; zlims = [9,19];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_002'; gene = 'uninj'; zlims = [16,25];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_004'; gene = 'uninj'; zlims = [11,23]; 

# inj files
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_002'; gene = 'inj'; zlims = [14,19];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_004'; gene = 'inj'; #EXCLUDE 
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L01_004_mip'; gene = 'inj'; #EXCLUDE
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L02_002'; gene = 'inj'; zlims = [15,19]; #no r mip, ignore or go back and get UV
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ inj_L03_002'; gene = 'inj'; zlims = [19,40];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L01_002'; gene = 'inj'; zlims = [24,31];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_002'; gene = 'inj'; zlims = [15,22];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L02_004'; gene = 'inj'; zlims = [11,19];
# filePath = '20240726_F0CRX2b_s1c_xOG_3dpf_ uninj_L03_002'; gene = 'inj'; zlims = [12,20];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L01_002'; gene = 'inj'; zlims = [12,21];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_002'; gene = 'inj'; zlims = [19,23];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L02_004'; gene = 'inj'; zlims = [7,16];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L03_002'; gene = 'inj'; zlims = [7,20];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L04_002'; gene = 'inj'; zlims = [14,20];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L05_002'; gene = 'inj'; zlims = [16,18];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L06_002'; gene = 'inj'; zlims = [14,17];
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_002'; gene = 'inj'; zlims = [13,17]; #no file directory
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L01_004'; gene = 'inj'; zlims = [13,25]; #no file directory
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_002'; gene = 'inj'; zlims = [3,27]; #no file directory
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L02_004'; gene = 'inj'; zlims = [13,18]; #no file directory
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_002'; gene = 'inj'; zlims = [19,22]; #no file directory
# filePath = '20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_004'; gene = 'inj'; zlims = [15,24]; #no file directory

# define subdirectory for analysis
dirOut = dirAnalysis + gene + '/' + filePath + '/'
print('Viewer cleared...')

# clear key binds
@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    ...

# load mips
mipO = 0.65
viewer.open(dirOut + layerNames[0] + "_mip.tiff", plugin='napari', colormap = 'green', blending='additive', opacity=mipO);
viewer.open(dirOut + layerNames[1] + "_mip.tiff", plugin='napari', colormap = 'magenta', blending='additive', opacity=mipO);
# viewer.open(dirOut + layerNames[2] + "_mip.tiff", plugin='napari', colormap = 'gray', blending='additive', opacity=mipO);

# load segmentation
if (os.path.isfile(dirOut + layerNames[0] + "_seg_curated.tiff")):
    print('Loaded previously curated segmentation for {0}'.format(layerNames[0]))
    viewer.open(dirOut + layerNames[0] + "_seg_curated.tiff", name=layerNames[0] + '_seg', plugin='napari',blending='additive');
else:
    segData = np.load(dirOut + layerNames[0] + "_mip" + "_seg.npy", allow_pickle=True).item()
    viewer.add_labels(segData['masks'], name= layerNames[0] + '_seg',blending='additive');
viewer.layers[layerNames[0] + '_seg'].preserve_labels = True;

if (os.path.isfile(dirOut + layerNames[1] + "_seg_curated.tiff")):
    print('Loaded previously curated segmentation for {0}'.format(layerNames[1]))
    viewer.open(dirOut + layerNames[1] + "_seg_curated.tiff", name=layerNames[1] + '_seg', plugin='napari',blending='additive');
else:
    segData = np.load(dirOut + layerNames[1] + "_mip" + "_seg.npy", allow_pickle=True).item()
    viewer.add_labels(segData['masks'], name= layerNames[1] + '_seg',blending='additive');
viewer.layers[layerNames[1] + '_seg'].preserve_labels = True;


viewer.layers[layerNames[0] + '_mip'].visible = False
# viewer.layers[layerNames[3] + '_mip'].visible = False
viewer.layers[layerNames[0] + '_seg'].visible = False

print('Loaded  ' + filePath + ' !\n Now fix it and save it!')

#define useful keyboard shortcuts

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
   lname = layerNames[1] + '_seg'
   tempd = viewer.layers[lname].data
   tempd[tempd == viewer.layers[lname].selected_label]=0
   viewer.layers[lname].data = tempd
   print('Cut!')
print('removeLabel on {0}_seg ("Shift-X")'.format(layerNames[1]))

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[1] + '_seg'
   if (viewer.layers[lname].preserve_labels == True):
      viewer.layers[lname].preserve_labels = False
   elif (viewer.layers[lname].preserve_labels == False):
      viewer.layers[lname].preserve_labels = True
print('toggle {0}_seg preserve_labels ("k")'.format(layerNames[1]))
        
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[1] + '_mip'
   if (viewer.layers[lname].visible == True):
      viewer.layers[lname].visible = False
   elif (viewer.layers[lname].visible == False):
      viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("B")'.format(layerNames[1]))

@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[0] + '_mip'
   if (viewer.layers[lname].visible == True):
      viewer.layers[lname].visible = False
   elif (viewer.layers[lname].visible == False):
      viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("N")'.format(layerNames[0]))

@viewer.bind_key('+', overwrite=True)
def new_label(viewer):
   """Set the currently selected label to the largest used label plus one."""
   lname = layerNames[1] + '_seg'
   viewer.layers[lname].selected_label = viewer.layers[lname].data.max() + 1
print('Add new label in {0}_seg visibility ("+"|"M")'.format(layerNames[1]))

Viewer cleared...


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Analysis\\F0CRXTbx2b\\xOGs1C\\inj\\20240726_F0CRX2b_s1c_xOG_5dpf_ uninj_L03_004\\R_mip.tiff'

In [310]:
# if needed, run this cell to reload whole z-stack
print('Loading data...')
nMIP = len(viewer.layers)
viewer.open((dirData+filePath + '.nd2'), plugin='napari-nikon-nd2')
viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();
if len(viewer.layers)==2+nMIP: #no DAPI
   viewer.layers[nMIP+0].colormap = 'green'
   viewer.layers[nMIP+0].name = layerNames[0]
   viewer.layers[nMIP+1].colormap = 'magenta'
   viewer.layers[nMIP+1].name = layerNames[1]
elif len(viewer.layers)==3+nMIP: #DAPI
   viewer.layers[nMIP+0].colormap = 'gray'
   viewer.layers[nMIP+0].name = 'N'
   viewer.layers[nMIP+0].opacity = 0.99
   viewer.layers[nMIP+1].colormap = 'green'
   viewer.layers[nMIP+1].name = layerNames[0]
   viewer.layers[nMIP+2].colormap = 'magenta'
   viewer.layers[nMIP+2].name = layerNames[1]
elif len(viewer.layers)==4+nMIP: #DAPI
   viewer.layers[nMIP+0].colormap = 'gray'
   viewer.layers[nMIP+0].name = 'N'
   viewer.layers[nMIP+0].opacity = 0.99
   viewer.layers[nMIP+1].colormap = 'green'
   viewer.layers[nMIP+1].name = layerNames[0]
   viewer.layers[nMIP+2].colormap = 'magenta'
   viewer.layers[nMIP+2].name = layerNames[1]
   viewer.layers[nMIP+3].colormap = 'bop orange'
   viewer.layers[nMIP+3].name = layerNames[2]

print('Images loaded!')

Loading data...
Images loaded!


#### resave first curated segmentation after napari-ing around

In [39]:
lname = layerNames[1] + '_seg'
l = viewer.layers[lname]; l.save(dirOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + filePath)

viewer.layers[layerNames[0] + '_mip'].visible = True
viewer.layers[layerNames[0] + '_seg'].visible = True
# viewer.layers[layerNames[0] + '_seg'].contour = 0
viewer.layers[layerNames[1] + '_mip'].visible = False
viewer.layers[layerNames[1] + '_seg'].visible = False

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
   lname = layerNames[0] + '_seg'
   tempd = viewer.layers[lname].data
   tempd[tempd == viewer.layers[lname].selected_label]=0
   viewer.layers[lname].data = tempd
   print('Cut!')
print('removeLabel on {0}_seg ("Shift-X")'.format(layerNames[0]))

@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[1] + '_mip'
   if (viewer.layers[lname].visible == True):
      viewer.layers[lname].visible = False
   elif (viewer.layers[lname].visible == False):
      viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("B")'.format(layerNames[1]))

@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[0] + '_mip'
   if (viewer.layers[lname].visible == True):
      viewer.layers[lname].visible = False
   elif (viewer.layers[lname].visible == False):
      viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("N")'.format(layerNames[0]))


@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[0] + '_seg'
   if (viewer.layers[lname].preserve_labels == True):
      viewer.layers[lname].preserve_labels = False
   elif (viewer.layers[lname].preserve_labels == False):
      viewer.layers[lname].preserve_labels = True
print('toggle {0}_seg preserve_labels ("k")'.format(layerNames[0]))

@viewer.bind_key('+', overwrite=True)
def new_label(viewer):
   """Set the currently selected label to the largest used label plus one."""
   lname = layerNames[0] + '_seg'
   viewer.layers[lname].selected_label = viewer.layers[lname].data.max() + 1
print('Add new label in {0}_seg visibility ("+"|"M")'.format(layerNames[0]))

Done with U_seg for 20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L06_002
removeLabel on R_seg ("Shift-X")
toggle U_mip visibility ("B")
toggle R_mip visibility ("N")
toggle R_seg preserve_labels ("k")
Add new label in R_seg visibility ("+"|"M")


#### resave second curated segmentation after napari-ing around

In [40]:
lname = layerNames[0] + '_seg'
l = viewer.layers[lname]; l.save(dirOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + filePath)

Done with R_seg for 20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L06_002


### Reload curated segmentations after saving for final review

In [41]:
# remove all except mips
viewer.layers.remove(viewer.layers[layerNames[1] + '_seg'])
viewer.layers.remove(viewer.layers[layerNames[0] + '_seg'])

viewer.open(dirOut + layerNames[1] + "_seg_curated.tiff", name=layerNames[1] + '_seg', plugin='napari',blending='additive');
# viewer.layers['U_seg'].contour = 5
viewer.layers[layerNames[1] + '_mip'].visible = True
viewer.open(dirOut + layerNames[0] + '_seg_curated.tiff', name=layerNames[0] + '_seg', plugin='napari',blending='additive');

print(filePath)
n0 = len(np.unique(viewer.layers[layerNames[1] + '_seg'].data))-1
print("Image has {0} {1}".format(n0, layerNames[1]))
n1 = len(np.unique(viewer.layers[layerNames[0] + '_seg'].data))-1
print("Image has {0} {1}".format(n1,layerNames[0]))

20240726_F0CRX2b_s1c_xOG_5dpf_ inj_L06_002
Image has 271 U
Image has 31 R


### [Go back and run next one &uarr;](#manualCuration)

In [55]:
layerNames[1]

'U'

In [55]:
layerNames[1]

'U'